In [15]:
#Mean is slightly better ~91% w/o gaussian angles
#Mean is slightly worse ~87% w/ gaussian angles
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import average, Input, Concatenate

#Import lee filter
import sys
sys.path.insert(0, '../Filters')
sys.path.insert(0, '../../Kevin')

from extra_functions import *
import mean

#Import date time
import datetime

def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = "../../train.json"
test_path =  "../../test.json"

train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)
#train_df.inc_angle = train_df.inc_angle.replace('na', 0)
#train_df.inc_angle = train_df.inc_angle.astype(float).fillna(0.0)
train_df.inc_angle = train_df.inc_angle.replace('na', np.random.normal(37.5, 7.5, 1)[0])
train_df.inc_angle = train_df.inc_angle.astype(float).fillna(np.random.normal(37.5, 7.5, 1)[0])
x_angle_train = np.array(train_df.inc_angle)
x_angle_test = np.array(test_df.inc_angle)   
y_train = to_categorical(train_df["is_iceberg"])

In [16]:
train_images[1,:,:,0]

array([[-12.242375, -14.920305, -14.920363, ..., -10.633402, -11.956054,
        -14.379026],
       [-13.046626, -14.077841, -12.728542, ..., -11.243189, -11.786291,
        -14.379026],
       [-15.248114, -14.0779  , -12.604514, ..., -15.680162, -13.51685 ,
        -14.454887],
       ..., 
       [-15.589596, -16.899685, -17.527912, ..., -11.78711 , -10.883234,
        -12.306804],
       [-14.761801, -15.503286, -15.333027, ...,  -9.793052,  -8.062263,
         -9.882715],
       [-13.178177, -15.503286, -14.375814, ..., -11.087559,  -8.246684,
         -8.666801]])

In [17]:
train_images[:,:,:,0] = mean.mean_filter_df(train_images[:, :, :, 0])

100%|██████████| 1604/1604 [00:53<00:00, 29.20it/s]
